In [77]:
import csv
import math

# TODO remove when not on google colab
from google.colab import files
uploaded = files.upload()

Saving dadosBenchmark_validacaoAlgoritmoAD.csv to dadosBenchmark_validacaoAlgoritmoAD.csv


In [138]:
import random

DISCRETE_ATT = 1
CONTINUOUS_ATT = 2

dataset = []
preditiveAttributes = {}
allAttributes = {}
attributesTypes = {}
EXPECTED_COLUMN = ""

def openFile(filename, delimiter, attTypes):
  with open(filename, 'r') as file:
    global attributesTypes, preditiveAttributes, allAttributes, dataset, EXPECTED_COLUMN
    lines = csv.reader(file, delimiter=delimiter)
    list_lines = list(lines)

    attributesTypes = attTypes
    preditiveAttributes = {x:i for i,x in enumerate(list_lines[0][:-1])}
    allAttributes = {x:i for i,x in enumerate(list_lines[0])} #Includes Joga

    dataset = list_lines[1:]
    EXPECTED_COLUMN = list_lines[0][-1]

    
openFile("diabetes.csv", ',', {'Pregnancies': CONTINUOUS_ATT, 'Glucose': CONTINUOUS_ATT, 'BloodPressure': CONTINUOUS_ATT, 'SkinThickness': CONTINUOUS_ATT, 'Insulin': CONTINUOUS_ATT, 'BMI': CONTINUOUS_ATT, 'DiabetesPedigreeFunction': CONTINUOUS_ATT, 'Age': CONTINUOUS_ATT, 'Outcome': DISCRETE_ATT})
#openFile("dadosBenchmark_validacaoAlgoritmoAD.csv", ';', {'Temperatura': DISCRETE_ATT, 'Tempo': DISCRETE_ATT, 'Umidade': DISCRETE_ATT, 'Ventoso': DISCRETE_ATT, 'Joga': DISCRETE_ATT})

print("dataset: " + str(dataset))
print("allAttributes:" + str(allAttributes))
print("preditiveAttributes:" + str(preditiveAttributes))
print("attributeTypes:" + str(attributesTypes))


dataset: [['6', '148', '72', '35', '0', '33.6', '0.627', '50', '1'], ['1', '85', '66', '29', '0', '26.6', '0.351', '31', '0'], ['8', '183', '64', '0', '0', '23.3', '0.672', '32', '1'], ['1', '89', '66', '23', '94', '28.1', '0.167', '21', '0'], ['0', '137', '40', '35', '168', '43.1', '2.288', '33', '1'], ['5', '116', '74', '0', '0', '25.6', '0.201', '30', '0'], ['3', '78', '50', '32', '88', '31', '0.248', '26', '1'], ['10', '115', '0', '0', '0', '35.3', '0.134', '29', '0'], ['2', '197', '70', '45', '543', '30.5', '0.158', '53', '1'], ['8', '125', '96', '0', '0', '0', '0.232', '54', '1'], ['4', '110', '92', '0', '0', '37.6', '0.191', '30', '0'], ['10', '168', '74', '0', '0', '38', '0.537', '34', '1'], ['10', '139', '80', '0', '0', '27.1', '1.441', '57', '0'], ['1', '189', '60', '23', '846', '30.1', '0.398', '59', '1'], ['5', '166', '72', '19', '175', '25.8', '0.587', '51', '1'], ['7', '100', '0', '0', '0', '30', '0.484', '32', '1'], ['0', '118', '84', '47', '230', '45.8', '0.551', '31', 

In [139]:
# Calculate entropy for given attribute
def entropy(dataset, attribute_name):
  
  attribute = allAttributes[attribute_name]
  
  frequency = {}
  attribute_entropy = 0;
  
  for line in dataset:
    if line[attribute] in frequency:
      frequency[line[attribute]] += 1.0
    else:
      frequency[line[attribute]] = 1.0
            
  for value in frequency.values():
    pi = value / len(dataset)
    attribute_entropy += - pi * math.log(pi,2)
  
  return round(attribute_entropy,5)

for attribute in allAttributes:
  print("Entropia para " + attribute + ": " + str(entropy(dataset, attribute)))


Entropia para Pregnancies: 3.48205
Entropia para Glucose: 6.75102
Entropia para BloodPressure: 4.79247
Entropia para SkinThickness: 4.58559
Entropia para Insulin: 4.68159
Entropia para BMI: 7.59376
Entropia para DiabetesPedigreeFunction: 8.8285
Entropia para Age: 5.02884
Entropia para Outcome: 0.93313


In [140]:

# Calculate gain for a given attribute
def gain(dataset, attribute_name):
  frequency = {} # frequency of a value 
  total_gain = entropy(dataset, EXPECTED_COLUMN) # entropy of default

  attribute = preditiveAttributes[attribute_name]
  
  for line in dataset:
    if line[attribute] in frequency:
      frequency[line[attribute]] += 1.0
    else:
      frequency[line[attribute]] = 1.0
      
  for k, v in frequency.items():
    pi = v / len(dataset)
    subset = [line for line in dataset if line[attribute] == k]
    total_gain -= pi * entropy(subset, EXPECTED_COLUMN)
      
  return round(total_gain,3)



for attribute in preditiveAttributes:
  print("Ganho para " + attribute + ": " + str(gain(dataset, attribute)))



Ganho para Pregnancies: 0.062
Ganho para Glucose: 0.304
Ganho para BloodPressure: 0.059
Ganho para SkinThickness: 0.082
Ganho para Insulin: 0.277
Ganho para BMI: 0.344
Ganho para DiabetesPedigreeFunction: 0.651
Ganho para Age: 0.141


In [0]:
#@title Define Tree Class
import operator

class Node(object):
  def __init__(self, attribute=None):
    self.attribute = attribute
    self.classType = None
    self.operator = operator.eq
    self.descendents = {}
    
  def addBranch(self, subAttribute, op, node):
    print("addBranch: adicionando para " + str(subAttribute) + str(op) + str(node))
    self.descendents[subAttribute] = node
    print("Nodo " + str(self) + " possui descendentes: " + str(self.descendents))
    #self.operator = op
    self.descendents[subAttribute].updateOperator(op)
    
  def updateClass(self, classType):
    self.classType = classType

  def updateAttribute(self, attribute):
    self.attribute = attribute
    
  def updateOperator(self, op):
    self.operator = op


In [142]:
def countClasses(dataset, attribute_name=EXPECTED_COLUMN):
  frequency = {} # frequency of a value

  attribute = allAttributes[attribute_name]
  
  for line in dataset:
    if line[attribute] in frequency:
      frequency[line[attribute]] += 1.0
    else:
      frequency[line[attribute]] = 1.0
      
  return frequency
      
countClasses(dataset)

{'0': 500.0, '1': 268.0}

In [0]:
def decisionTree(dataset, attributes):
  #print("decisionTree() chamada para dataset e atributos: " + str(attributes))
  # Crie um nó N
  node = Node()

  datasetClasses = countClasses(dataset)
  
  #print("decisionTree() attributes: " + str(attributes))

  # Se todos os exemplos em D possuem a mesma classe yi, então retorne N como um nó folha rotulado com y
  if len(datasetClasses) == 1:
    
    #print("Todos exemplos com mesma classe " + str(dataset) + str(attributes))
    node.updateClass(list(countClasses(dataset).keys())[0])
    return node

  # Se L é vazia, então retorne N como um nó folha rotulado com a classe yi mais frequente em D
  elif len(attributes) == 0:

    #print("L vazia")
    higherFrequency = -1

    for key in datasetClasses:
      if datasetClasses[key] > higherFrequency:
        higherFrequency = datasetClasses[key]

    node.updateClass(list(countClasses(dataset).keys())[0])
    return node

  else:
    #print("Achar melhor criterio de divisão " + str(attributes))
    # A = atributo preditivo em L que apresenta "melhor" critério de divisão
    bestAttribute = random.sample(attributes.keys(), 1)[0]
    bestAttributeGain = 0
    randomAttributes = random.sample(attributes.keys(), min(int(math.sqrt(len(preditiveAttributes))), len(attributes)))
    #print("random attributes= " + str(randomAttributes))
    for attribute in randomAttributes:
      currentAttributeGain = gain(dataset, attribute)
      if currentAttributeGain > bestAttributeGain:
        bestAttribute = attribute
        bestAttributeGain = currentAttributeGain
        #print("-> new best attribute: " + str(attribute))
        #print("-> new gain: " + str(gain(dataset, attribute)))

    # Associe A ao nó N
    node.updateAttribute(bestAttribute)

    # L = L - A
    del attributes[bestAttribute]

    #Para cada valor v distinto do atributo A, considerando os exemplos em D, faça
    comparableAttributes = []       # lista com nome do atributo e operador
    if attributesTypes[bestAttribute] == DISCRETE_ATT:
      for i in countClasses(dataset, bestAttribute).keys():
        comparableAttributes.append([i, operator.eq])
      #print(str(comparableAttributes))
    else :
      average = sum(float(line[preditiveAttributes[bestAttribute]]) for line in dataset) / len(dataset)
      comparableAttributes.append([average+0.1, operator.le]) # TODO MUDAR AQUIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII
      comparableAttributes.append([average-0.1, operator.gt])
    
    for subAttribute in comparableAttributes: 
      #print(str(subAttribute))
      #Dv = subconjunto dos dados de treinamento em que A = v
      subDataset = []
      
      value, op = subAttribute
      
      for line in dataset:
        if op == operator.eq: # DISCRETE_ATTRIBUTE
          if line[preditiveAttributes[bestAttribute]] == value:
            subDataset.append(line)  
        else: # CONTINUOUS ATTRIBUTE
          if op(float(line[preditiveAttributes[bestAttribute]]), value):
            #print("Average: " + str(value) + " - number: " + str(line[preditiveAttributes[bestAttribute]]))
            subDataset.append(line)
      
      #Se Dv vazio, então retorne N como um nó folha rotulado com a classe yi mais frequente em Dv. 
      if len(subDataset) == 0:
        print("Dv vazio")
        subDatasetClasses = countClasses(subDataset)
        
        higherFrequency = -1

        for key in subDatasetClasses:
          if subDatasetClasses[key] > higherFrequency:
            higherFrequency = subDatasetClasses[key]

        node.updateClass(list(countClasses(dataset).keys())[0])
        #return node
      
      #Senão, associe N a uma subárvore retornada por arvoreDeDecisao(Dv,L)    
      else: 
        #print("\n")
        #print(value)
        #print(op)
        node.addBranch(value, op, decisionTree(subDataset, attributes))
        #newnode = node.addBranch(value, op, decisionTree(subDataset, attributes))
        #newnode = decisionTree(subDataset, attributes)
        
    #retorne N
    return node
      
      
  
  

In [160]:
import copy

copyPreditiveAttributes = copy.copy(preditiveAttributes)
root_node = decisionTree(dataset, copyPreditiveAttributes)


addBranch: adicionando para 97.63731343283581<built-in function le><__main__.Node object at 0x7f1b6c4baa90>
Nodo <__main__.Node object at 0x7f1b6c4baa58> possui descendentes: {97.63731343283581: <__main__.Node object at 0x7f1b6c4baa90>}
addBranch: adicionando para 8.1<built-in function le><__main__.Node object at 0x7f1b6c4ba8d0>
Nodo <__main__.Node object at 0x7f1b6c4ba9e8> possui descendentes: {8.1: <__main__.Node object at 0x7f1b6c4ba8d0>}
addBranch: adicionando para 7.9<built-in function gt><__main__.Node object at 0x7f1b6c4baa20>
Nodo <__main__.Node object at 0x7f1b6c4ba9e8> possui descendentes: {8.1: <__main__.Node object at 0x7f1b6c4ba8d0>, 7.9: <__main__.Node object at 0x7f1b6c4baa20>}
addBranch: adicionando para 62.01428571428571<built-in function le><__main__.Node object at 0x7f1b6c4ba9e8>
Nodo <__main__.Node object at 0x7f1b6c4ba908> possui descendentes: {62.01428571428571: <__main__.Node object at 0x7f1b6c4ba9e8>}
addBranch: adicionando para 61.81428571428571<built-in functi

In [161]:
def printTree(node, level=0):
  print(''.join(['   ' for s in range(level)]) + str(node.attribute) + ": " + str(node.classType))
  for k, v in node.descendents.items():
    print(''.join(['   ' for s in range(level)]) + '-> ' + str(k))
    printTree(v, level + 1)
    
printTree(root_node)
print(str(root_node))
print(str(root_node.descendents))

Pregnancies: None
-> 3.9450520833333336
   Insulin: None
   -> 89.22264150943396
      DiabetesPedigreeFunction: None
      -> 0.5531124031007754
         BMI: None
         -> 29.28142076502733
            Age: None
            -> 25.72637362637363
               Glucose: None
               -> 97.63731343283581
                  None: 0
               -> 97.43731343283582
                  BloodPressure: None
                  -> 62.01428571428571
                     SkinThickness: None
                     -> 8.1
                        None: 0
                     -> 7.9
                        None: 0
                  -> 61.81428571428571
                     None: 0
            -> 25.526373626373626
               None: 0
         -> 29.081420765027328
            None: 1
      -> 0.3531124031007754
         None: 1
   -> 89.02264150943397
      None: 0
-> 3.7450520833333334
   None: 1
{3.9450520833333336: <__main__.Node object at 0x7f1b6c4b22e8>, 3.7450520833333334: <__main__.

In [0]:
def classify(node, entry):
  
  if not node.descendents:
    return node.classType

  for value, desc_node in node.descendents.items():
    op = desc_node.operator
    
    if ((op == operator.eq and entry[preditiveAttributes[node.attribute]] == value) or (op(float(entry[preditiveAttributes[node.attribute]]), value))):
      return classify(desc_node, entry)
  
  return None
  

In [174]:
example = dataset[5]
print(example)
example = dataset[5][:-1]

print("Resultado: " + str(classify(root_node, example)))
#TODO as vezes ta retornando None pq a arvore foi montada com um conjunto q nao tinha algum tipo de atributo. O que se faz, se descarta none?

['5', '116', '74', '0', '0', '25.6', '0.201', '30', '0']
Resultado: 1


In [0]:
ntree = 10
import random

def createBootstrap(dataset, fraction):
  bootstrap = random.sample(dataset, int(len(dataset)*fraction))
  bootstrap += (random.sample(bootstrap, len(dataset) - len(bootstrap)))
  
  #TODO precisa criar testing set?
  return bootstrap

In [175]:

def createForest():
  forest = []
  for i in range(ntree):
    bootstrap = createBootstrap(dataset, fraction=0.66)
    copyPreditiveAttributes = copy.copy(preditiveAttributes)
    tree = decisionTree(bootstrap, copyPreditiveAttributes)
    forest.append(tree)
  
  return forest

def most_common(lst):
    return max(set(lst), key=lst.count)

def main():
    forest = createForest();
    
    testSet = [dataset[9][:-1], dataset[10][:-1]]
    for test in testSet:
      results = []
      for tree in forest:
        answer = classify(tree, test)
        if answer != None:
          results.append(answer)
      
      print(results)
      print("Votação majoritária: " + str(most_common(results)))

main()

addBranch: adicionando para 25.83913043478261<built-in function le><__main__.Node object at 0x7f1b6c449780>
Nodo <__main__.Node object at 0x7f1b6c449710> possui descendentes: {25.83913043478261: <__main__.Node object at 0x7f1b6c449780>}
addBranch: adicionando para 78.52857142857142<built-in function le><__main__.Node object at 0x7f1b6c449940>
Nodo <__main__.Node object at 0x7f1b6c4498d0> possui descendentes: {78.52857142857142: <__main__.Node object at 0x7f1b6c449940>}
addBranch: adicionando para 21.433333333333334<built-in function le><__main__.Node object at 0x7f1b6c449a58>
Nodo <__main__.Node object at 0x7f1b6c4497b8> possui descendentes: {21.433333333333334: <__main__.Node object at 0x7f1b6c449a58>}
addBranch: adicionando para 21.23333333333333<built-in function gt><__main__.Node object at 0x7f1b6c449860>
Nodo <__main__.Node object at 0x7f1b6c4497b8> possui descendentes: {21.433333333333334: <__main__.Node object at 0x7f1b6c449a58>, 21.23333333333333: <__main__.Node object at 0x7f1